# 유저데이터 추가 수집
- 중복 제거
- 스팀아이디만 누락 없으면 크롤링 진행 가능, 스팀아이디 누락 없는거 확인

In [21]:
import pandas as pd
df = pd.read_csv('최종_유저데이터2차.csv')
df

,SteamID64,작성일,본문,플레이 시간,추천 여부,계정 레벨,게임 목록
0,76561198089026294,2025-08-04,really bad,4.6 hrs on record,0,109.0,"Counter-Strike, Counter-Strike: Condition Zero..."
1,76561198840751916,2025-08-04,wen game,34.2 hrs on record,1,8.0,"Left 4 Dead 2, Worms Ultimate Mayhem, The Walk..."
2,76561199758331866,2025-08-04,ww,14.3 hrs on record,1,1.0,NaN
3,76561198309660819,2025-08-04,"Really annoying sometimes, really fun sometimes",0.2 hrs on record,1,13.0,"Left 4 Dead 2, Terraria, Neverwinter, PlanetSi..."
4,76561199125555677,2025-08-04,I love this balanced game and matchmaking is a...,667.5 hrs on record,0,6.0,"Terraria, The Forest, Trove, DRAGON BALL XENOV..."
...,...,...,...,...,...,...,...
12566,76561198401068937,2025-07-31,your server down by itself and i got suspended...,304.6 hrs on record,0,11.0,NaN
12567,76561199006479136,2025-07-31,praise,301.3 hrs on record,1,NaN,NaN
12568,76561199719259546,2025-07-30,cool,13.8 hrs on record,1,1.0,NaN
12569,76561198450677497,2025-07-30,Large level gap,112.7 hrs on record,0,6.0,"Life is Strange™, World of Tanks Blitz, Counte..."


In [22]:
df['SteamID64'].duplicated().sum()

np.int64(71)

In [23]:
df.drop_duplicates(subset=['SteamID64'], inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12500 entries, 0 to 12517
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SteamID64  12500 non-null  object 
 1   작성일        12500 non-null  object 
 2   본문         12499 non-null  object 
 3   플레이 시간     12500 non-null  object 
 4   추천 여부      12500 non-null  int64  
 5   계정 레벨      10974 non-null  float64
 6   게임 목록      1994 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 781.2+ KB


In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# -------------------- 크롬 브라우저 설정 --------------------
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

df['최근플레이시간'] = '-'
df['총리뷰수'] = '-'
df['최근게임1'] = '-'
df['최근게임2'] = '-'
df['최근게임3'] = '-'

for id in df['SteamID64'] :
    url = f'https://steamcommunity.com/profiles/{id}'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    playtime = soup.find('div', 'recentgame_quicklinks recentgame_recentplaytime').text.strip('\n')

    review_section = soup.find('span', string='Reviews')
    if review_section:
        parent = review_section.find_parent('a')  # <a> 태그
        review_count_tag = parent.find('span', class_='profile_count_link_total')
        if review_count_tag:
            review_count = review_count_tag.text.strip()
        else :
            review_count = '데이터 없음'
    else : 
        review_count = '데이터 없음'

    name = []
    data = soup.find('div', 'recent_games')
    for i in data.find_all('div', 'game_name') :
        name.append(i)


In [ ]:
# 최근 2주간 플레이타임(스팀)
soup.find('div', 'recentgame_quicklinks recentgame_recentplaytime').text.strip('\n')

'49.8 hours past 2 weeks'

In [15]:
from bs4 import BeautifulSoup

# soup는 해당 HTML 페이지 전체를 파싱한 BeautifulSoup 객체라고 가정합니다
review_section = soup.find('span', string='Reviews')

if review_section:
    parent = review_section.find_parent('a')  # <a> 태그
    review_count_tag = parent.find('span', class_='profile_count_link_total')
    if review_count_tag:
        review_count = review_count_tag.text.strip()
        print(f"리뷰 수: {review_count}")
    else:
        print("리뷰 수를 찾을 수 없음")
else:
    print("리뷰 섹션이 없음")


리뷰 수: 1


In [19]:
data = soup.find('div', 'recent_games')
name = data.find_all('div', 'game_name')
for i in name :
    print(i.text.strip())

R.E.P.O.
Overwatch® 2
VRChat


In [25]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# -------------------- 브라우저 설정 --------------------
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option('detach', True)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# -------------------- 데이터프레임 설정 --------------------
df['최근플레이시간'] = '-'
df['총리뷰수'] = '-'
df['최근게임1'] = '-'
df['최근게임2'] = '-'
df['최근게임3'] = '-'

save_interval = 10  # 10명마다 저장
output_file = '중간저장_steam데이터.csv'

for idx, steam_id in enumerate(df['SteamID64']):
    print(f"{idx+1}/{len(df)}: {steam_id} 처리 중...")

    url = f'https://steamcommunity.com/profiles/{steam_id}'
    try:
        driver.get(url)
        time.sleep(1.5)  # 로딩 대기

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 최근 플레이 시간
        try:
            playtime_tag = soup.find('div', class_='recentgame_quicklinks recentgame_recentplaytime')
            playtime = playtime_tag.text.strip() if playtime_tag else '없음'
        except:
            playtime = '없음'

        # 리뷰 수
        try:
            review_section = soup.find('span', string='Reviews')
            if review_section:
                parent = review_section.find_parent('a')
                review_count_tag = parent.find('span', class_='profile_count_link_total')
                review_count = review_count_tag.text.strip() if review_count_tag else '데이터 없음'
            else:
                review_count = '데이터 없음'
        except:
            review_count = '데이터 없음'

        # 최근 게임 이름 3개
        game_names = ['-', '-', '-']
        try:
            recent_games = soup.find('div', class_='recent_games')
            if recent_games:
                names = recent_games.find_all('div', class_='game_name')
                for i in range(min(3, len(names))):
                    game_names[i] = names[i].text.strip()
        except:
            pass

        # 결과 출력
        print(f"▶ 최근 플레이 시간: {playtime}")
        print(f"▶ 총 리뷰 수: {review_count}")
        print(f"▶ 최근 게임 1: {game_names[0]}")
        print(f"▶ 최근 게임 2: {game_names[1]}")
        print(f"▶ 최근 게임 3: {game_names[2]}")
        print("-" * 40)

        # 데이터프레임에 저장
        df.at[idx, '최근플레이시간'] = playtime
        df.at[idx, '총리뷰수'] = review_count
        df.at[idx, '최근게임1'] = game_names[0]
        df.at[idx, '최근게임2'] = game_names[1]
        df.at[idx, '최근게임3'] = game_names[2]

        # 중간 저장
        if (idx + 1) % save_interval == 0:
            df.to_csv(output_file, index=False)
            print(f"{idx+1}명 처리 후 중간 저장 완료")

    except Exception as e:
        print(f"오류 발생 - SteamID: {steam_id}, 오류: {e}")
        continue

# 최종 저장
df.to_csv('최종_Steam데이터.csv', index=False)
print("✅ 전체 작업 완료 및 저장 완료")


1/12500: 76561198089026294 처리 중...
▶ 최근 플레이 시간: 30.1 hours past 2 weeks
▶ 총 리뷰 수: 30
▶ 최근 게임 1: Overwatch® 2
▶ 최근 게임 2: Apex Legends
▶ 최근 게임 3: Dead by Daylight
----------------------------------------
2/12500: 76561198840751916 처리 중...
▶ 최근 플레이 시간: 49.8 hours past 2 weeks
▶ 총 리뷰 수: 1
▶ 최근 게임 1: R.E.P.O.
▶ 최근 게임 2: Overwatch® 2
▶ 최근 게임 3: VRChat
----------------------------------------
3/12500: 76561199758331866 처리 중...
▶ 최근 플레이 시간: 없음
▶ 총 리뷰 수: 1
▶ 최근 게임 1: -
▶ 최근 게임 2: -
▶ 최근 게임 3: -
----------------------------------------
4/12500: 76561198309660819 처리 중...
▶ 최근 플레이 시간: 없음
▶ 총 리뷰 수: 5
▶ 최근 게임 1: -
▶ 최근 게임 2: -
▶ 최근 게임 3: -
----------------------------------------
5/12500: 76561199125555677 처리 중...
▶ 최근 플레이 시간: 67.4 hours past 2 weeks
▶ 총 리뷰 수: 4
▶ 최근 게임 1: Counter-Strike 2
▶ 최근 게임 2: Overwatch® 2
▶ 최근 게임 3: Don't Starve
----------------------------------------
6/12500: 76561199068248962 처리 중...
▶ 최근 플레이 시간: 없음
▶ 총 리뷰 수: 데이터 없음
▶ 최근 게임 1: -
▶ 최근 게임 2: -
▶ 최근 게임 3: -
-----------------